### James Butcher - 5-18-25

A basic test setup for LangGraph using Python and Ollama running locally

## Basic imports

In [1]:
import os

## LLM

In [2]:
from langchain_ollama import ChatOllama

model_path = os.path.expanduser(
    "~/.cache/lm-studio/models/lmstudio-community/"
    "Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"
)

""" API Docs: 
https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html
"""
llm = ChatOllama(
    model="llama3.2",
    model_path=model_path,
    chat_format="chatml", # Works with most models
    temperature=1.0,  # Controlds the randomness/creativity [0.0 - 2.0]
    num_predict=64,  # Max number of tokens
    #n_ctx=2048, # Context length - default: 2048 (others are 512, 1024, 4096)
    #n_threads=4, # Number of threads to use for inference (default is 4)
    #num_predict=128  # Maximum number of tokens to predict when generating text. (Default: 128, -1 = infinite generation, -2 = fill context)
)

## Types and Fields imports

In [3]:
from typing import Annotated, Literal
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

## LangGraph Imports

In [4]:
from langgraph.graph import StateGraph, START, END  # For building the graph
from langgraph.graph.message import add_messages    # For defining states

## Define States

Define the structure of the "States" to be used by the graph
The below definition forces states to be of this form:
```
{ 
    "messages": [ <messages list> ]
}
```

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

## Define Nodes (Actions)

Define a node by creating a function that takes a state and returns a modified state.

In [6]:
def chatbot(state: State):
    # Take all the messages from the previous state and pass it to the LLM.
    # Then set the new set of messages to the response.
    return {"messages": [llm.invoke(state["messages"])]}

## Build the Graph

In [7]:
graph_builder = StateGraph(State)

#### Add all the nodes defined above

In [8]:
graph_builder.add_node("chatbot", chatbot)
# ... add more here ...

#### Connect all the nodes
- All graphs require a start and end node.
```
  START
    |
   \|/
[chatbot]
    |
   \|/
   END
```

In [9]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

#### Compile the graph

In [10]:
graph = graph_builder.compile()

# Usage

In [11]:
user_input = input("\nEnter a message: ")

starting_message = {
    "role": "user",
    "content": user_input
}
state = graph.invoke({"messages": [starting_message]})

print("\nOutput:" + state["messages"][-1].content)


Enter a message:  Write a bad poem



Output:Here's a terrible poem:

Rhubarb is red and squishy too
It's good for jam or so I chew
Pineapple pizza is the best
I like it on my shirt, it passes the test

My dog's name is Fluffy McFlufferson
He chases his tail
